# Linear Genetic Programming Based Approach for Robotic Controllers
In this lab session, we will leverage Linear Genetic Programming (LGP) to automatically design the controller for a mobile robot. Starting from a certain initial position (S), the robot must be able to navigate inside a maze until it reaches its home base (G). The maze consists of empty cells in which the robot can navigate freely and wall cells where the robot cannot pass. A ```Maze``` object also contains information about the optimal path to the goal, which can be used to evaluate the robot's navigation. 

In [2]:
import enum
import math
import random
import sys

sys.path.insert(0, "./..")

from utilities.robot_maze import Maze, Robot

In [3]:
cellcodes = enum.Enum('cellcodes', 'EMPTY WALL START ROUTE GOAL')

#build a maze
maze_list = [
        [cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.EMPTY, cellcodes.START, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.WALL, cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.ROUTE, cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.EMPTY, cellcodes.EMPTY],
        [cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL],
        [cellcodes.WALL, cellcodes.ROUTE, cellcodes.EMPTY, cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE],
        [cellcodes.EMPTY, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.ROUTE, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL, cellcodes.ROUTE],
        [cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.GOAL],
    ]

maze = Maze(maze_list, cellcodes)

print(maze)

            #     #     #  
#     #  #  #     S     #  
#        #  .  .  .     #  
.  .  .  #  .  #  #     #  
.  #  .  .  .  #  #        
.  .  #  #  #  #  #  #  #  
#  .     #  .  .  .  .  .  
   .  #  #  .  #     #  .  
#  .  .  .  .  #  #  #  G  



The robot can move in 3 possible ways:

- Move forward;
- Turn left;
- Turn right;

It can perceive the environment through 6 sensors that, when activated (value 1), indicate the presence of a wall in the corresponding position.

![Robot's sensors](../img/sensors.png "Robot's sensors")

Our controller must map the boolean values coming from the sensors into an action for the robot to take.

Now it's time to code! Define a set of operators which can be included in the program that controls the robot actions.

In [21]:
movecodes = enum.Enum('movecodes', 'FORWARD LEFT RIGHT')
opcodes = 

In [ ]:
# Generate a random program of length n
# You can tune the probability to have an actual move at a certain position with move_p
def random_program(n, move_p=0.3):
  prg = []
  moves = list(movecodes)
  func = list(opcodes)
  for _ in range(n):
    if random.random() < move_p:
      op = random.choice(moves)
    else:
      op = random.choice(func)
    prg.append(op)
  return prg

In [23]:
rp = random_program(10)

Complete the ```Robot``` class into the ```utilities/robot_maze.py``` file with a proper ```eval``` function.

In [24]:
r = Robot(rp, maze, maxMoves=70, movecodes=movecodes, opcodes=opcodes)

Define a fitness function for our navigation task. Hint: you can exploit the ```getRoute``` method of the ```Robot``` class and the ```scoreRoute``` method of the ```Maze``` class to get an estimate of the "correct steps" taken by the robot.

In [25]:
def fit(prg):
  # CODE HERE
  pass

As usual, the selection strategy we choose is tournament selection.

In [ ]:
def tournament_selection(fit, pop, t_size=4):
  tournament = random.choices(pop, k=t_size)
  return max(tournament, key=fit)

Implement functions for crossover and mutation.

In [27]:
def crossover(x, y):
  # CODE HERE
  pass

In [28]:
def mutation(x, p_m, move_p=0.3):
  # CODE HERE
  pass

Finally, we can implement a ```linear_GP``` function, using the functions defined above.

In [ ]:
def linear_GP(fit, pop_size, n_iter = 50):
  p_m = 0.1
  pop = [random_program(20) for _ in range(0, pop_size)]
  best = []
  for i in range(0, n_iter):
    selected = [tournament_selection(fit, pop) for _ in range(0, pop_size)]
    pairs = zip(selected, selected[1:] + [selected[0]])
    offsprings = []
    for x, y in pairs:
      of1, of2 = crossover(x, y)
      offsprings.append(of1)
      offsprings.append(of2)
    pop = [mutation(x, p_m) for x in offsprings]
    candidate_best = max(pop, key=fit)
    if fit(candidate_best) > fit(best):
      best = candidate_best
    # print(f"Best individual at generation {i}: {best}")
    print(f"Best fitness at generation {i}: {fit(best)}")
  return best

In [ ]:
random.seed(0)
best = linear_GP(fit, 1000)

In [ ]:
random.seed(0)
best = linear_GP(fit, 1000)

In [ ]:
best_robot = Robot(best, maze, 70, movecodes, opcodes)
best_robot.run()
print(best_robot.getRoute())

Are the robot moves consistent with the expected behaviour? (The robot starts with a south heading, hence towards the bottom of the monitor)

In [ ]:
print(maze)
best_robot.moves